In [1]:
from pathlib import Path
from typing import List, Dict, Any

import pandas as pd
from dotenv import load_dotenv


In [2]:
cwd = Path.cwd()
candidates = [cwd, cwd.parent, cwd.parent.parent]

project_root = None
for c in candidates:
    if (c / "data").exists() and (c / "src").exists():
        project_root = c
        break

if project_root is None:
    project_root = cwd.parent

print("Detected project root:", project_root)

env_path = project_root / ".env"
if env_path.exists():
    load_dotenv(env_path)

DATA_DIR = project_root / "data"
PROCESSED_DIR = DATA_DIR / "processed"

mcq_csv = PROCESSED_DIR / "langgraph_mcqs_dummy.csv"
if not mcq_csv.exists():
    raise FileNotFoundError(
        f"{mcq_csv} not found. Export MCQs from Notebook 4 before running this notebook."
    )

mcq_df = pd.read_csv(mcq_csv)
print("Loaded MCQs for evaluation:", mcq_csv)
print("Shape:", mcq_df.shape)
display(mcq_df.head(5))


Detected project root: C:\Users\Admin\OneDrive\Desktop\Capstone-MAT496
Loaded MCQs for evaluation: C:\Users\Admin\OneDrive\Desktop\Capstone-MAT496\data\processed\langgraph_mcqs_dummy.csv
Shape: (4, 8)


,id,stem,options,correct_option_index,difficulty,topic,source_excerpt,source_doc_id
0,probability distribution_mcq_1,Dummy question about probability distribution (1),"['Option A', 'Option B', 'Option C', 'Option D']",0,easy,probability distribution,Dummy excerpt 1.,dummy:page=0:chunk=0
1,probability distribution_mcq_2,Dummy question about probability distribution (2),"['Option A', 'Option B', 'Option C', 'Option D']",1,medium,probability distribution,Dummy excerpt 2.,dummy:page=0:chunk=1
2,probability distribution_mcq_1,Dummy question about probability distribution (1),"['Option A', 'Option B', 'Option C', 'Option D']",0,easy,probability distribution,Dummy excerpt 1.,dummy:page=0:chunk=0
3,probability distribution_mcq_2,Dummy question about probability distribution (2),"['Option A', 'Option B', 'Option C', 'Option D']",1,medium,probability distribution,Dummy excerpt 2.,dummy:page=0:chunk=1


In [3]:
print("Questions per topic:")
topic_counts = mcq_df["topic"].value_counts()
display(topic_counts.to_frame(name="num_questions"))


Questions per topic:


,num_questions
topic,
probability distribution,4


In [5]:
print("Questions per difficulty:")
if "difficulty" in mcq_df.columns:
    diff_counts = mcq_df["difficulty"].value_counts()
    display(diff_counts.to_frame(name="num_questions"))



Questions per difficulty:


,num_questions
difficulty,
easy,2
medium,2


In [6]:
def check_mcq_quality(row: pd.Series) -> Dict[str, Any]:
    """Return a dict of boolean flags + messages for one MCQ."""
    issues = []

    stem = str(row.get("stem", "")).strip()
    options = row.get("options", "")
    correct_idx = row.get("correct_option_index", None)

    if isinstance(options, str):
        # Very simple parser: assume list-like string from repr.
        if options.startswith("[") and options.endswith("]"):
            # Remove brackets and split on comma
            raw_items = options[1:-1].split(",")
            options_list = [item.strip().strip("'").strip('"') for item in raw_items]
        else:
            options_list = [options]
    else:
        options_list = list(options)

    if not stem:
        issues.append("empty_stem")

    if len(options_list) < 2:
        issues.append("too_few_options")

    try:
        idx = int(correct_idx)
        if idx < 0 or idx >= len(options_list):
            issues.append("correct_index_out_of_range")
    except Exception:
        issues.append("correct_index_not_int")

    if len(options_list) != len(set(options_list)):
        issues.append("duplicate_options")

    return {
        "has_issue": len(issues) > 0,
        "issues": ",".join(issues),
        "num_options": len(options_list),
    }


In [7]:
quality_rows = []
for _, row in mcq_df.iterrows():
    q = check_mcq_quality(row)
    quality_rows.append(q)

quality_df = pd.DataFrame(quality_rows)
eval_df = pd.concat([mcq_df, quality_df], axis=1)

display(eval_df.head(5))


,id,stem,options,correct_option_index,difficulty,topic,source_excerpt,source_doc_id,has_issue,issues,num_options
0,probability distribution_mcq_1,Dummy question about probability distribution (1),"['Option A', 'Option B', 'Option C', 'Option D']",0,easy,probability distribution,Dummy excerpt 1.,dummy:page=0:chunk=0,False,,4
1,probability distribution_mcq_2,Dummy question about probability distribution (2),"['Option A', 'Option B', 'Option C', 'Option D']",1,medium,probability distribution,Dummy excerpt 2.,dummy:page=0:chunk=1,False,,4
2,probability distribution_mcq_1,Dummy question about probability distribution (1),"['Option A', 'Option B', 'Option C', 'Option D']",0,easy,probability distribution,Dummy excerpt 1.,dummy:page=0:chunk=0,False,,4
3,probability distribution_mcq_2,Dummy question about probability distribution (2),"['Option A', 'Option B', 'Option C', 'Option D']",1,medium,probability distribution,Dummy excerpt 2.,dummy:page=0:chunk=1,False,,4


In [8]:
total = len(eval_df)
num_with_issues = eval_df["has_issue"].sum()

print(f"Total MCQs: {total}")
print(f"MCQs with any issue: {num_with_issues} ({num_with_issues/total:.1%})")

print("\nIssue type counts:")
issue_counts = (
    eval_df["issues"]
    .replace("", pd.NA)
    .dropna()
    .str.split(",", expand=True)
    .stack()
    .value_counts()
)
display(issue_counts.to_frame(name="count"))


Total MCQs: 4
MCQs with any issue: 0 (0.0%)

Issue type counts:


,count


In [9]:
eval_out_path = PROCESSED_DIR / "mcq_evaluation_results.csv"
eval_df.to_csv(eval_out_path, index=False, encoding="utf-8")
print("Saved full evaluation results to:", eval_out_path)


Saved full evaluation results to: C:\Users\Admin\OneDrive\Desktop\Capstone-MAT496\data\processed\mcq_evaluation_results.csv


In [10]:
clean_df = eval_df[~eval_df["has_issue"]].copy()
clean_out_path = PROCESSED_DIR / "mcq_clean_only.csv"
clean_df.to_csv(clean_out_path, index=False, encoding="utf-8")
print("Saved MCQs without rule-based issues to:", clean_out_path)
print("Clean MCQs count:", len(clean_df))


Saved MCQs without rule-based issues to: C:\Users\Admin\OneDrive\Desktop\Capstone-MAT496\data\processed\mcq_clean_only.csv
Clean MCQs count: 4


In [11]:
sample_size = min(5, len(eval_df))
sample = eval_df.sample(sample_size, random_state=42)

print(f"Random sample of {sample_size} questions for manual review:\n")
display(sample[["id", "stem", "options", "correct_option_index", "topic", "difficulty", "issues"]])


Random sample of 4 questions for manual review:



,id,stem,options,correct_option_index,topic,difficulty,issues
1,probability distribution_mcq_2,Dummy question about probability distribution (2),"['Option A', 'Option B', 'Option C', 'Option D']",1,probability distribution,medium,
3,probability distribution_mcq_2,Dummy question about probability distribution (2),"['Option A', 'Option B', 'Option C', 'Option D']",1,probability distribution,medium,
0,probability distribution_mcq_1,Dummy question about probability distribution (1),"['Option A', 'Option B', 'Option C', 'Option D']",0,probability distribution,easy,
2,probability distribution_mcq_1,Dummy question about probability distribution (1),"['Option A', 'Option B', 'Option C', 'Option D']",0,probability distribution,easy,
